In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from scipy.stats import spearmanr
import pandas as pd
import requests
from bs4 import BeautifulSoup
import dcor
import plotly.express as px
import matplotlib.pyplot as plt


In [2]:
tables=pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
sp500_wiki=tables[0]
sp500_wiki=sp500_wiki.iloc[:,:3]
print(sp500_wiki.shape)
sp500_wiki.head()

(503, 3)


,Symbol,Security,GICS Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ACN,Accenture,Information Technology


In [17]:
tickers=sp500_wiki['Symbol'].unique().tolist()
tickers[:10]

In [18]:
tickers = [t.replace('BF.B', 'BF-B').replace('BRK.B', 'BRK-B') for t in tickers]

In [19]:
start_date="2018-01-01"
end_date="2024-12-31"
df = yf.download(tickers  ,start=start_date, end=end_date)
df= df['Close'].reset_index()
print(df.shape)
df.head()

[*********************100%***********************]  8 of 10 completed

10 Failed downloads:
[*********************100%***********************]  8 of 10 completed['ABBV', 'AES', 'AOS', 'ABT', 'A', 'AFL', 'ACN', 'AMD', 'MMM', 'ADBE']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


(0, 11)


Ticker,Date,A,ABBV,ABT,ACN,ADBE,AES,AFL,AMD,AOS,MMM


In [6]:
df_low=df.isna().sum().reset_index()
df_low=df_low[df_low[0]>0]
df_low.head()

,Ticker,0


In [7]:
df_low['Ticker'].unique()

array([], dtype=object)

In [8]:
df = df.drop(columns=df_low['Ticker'].unique())
print(df.shape)
df.head()

(0, 504)


Ticker,Date,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS


In [9]:
sp500 = yf.download('^GSPC', start=start_date, end=end_date)
sp500= sp500['Close'].reset_index()
df=df.merge(sp500, how='inner')
print(df.shape)
df.head()

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['^GSPC']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


(0, 505)


Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,Date,^GSPC


In [10]:
#df.drop(columns=['Date'],axis=1,inplace=True)
df.set_index('Date', inplace=True)

df_returns = np.log(df / df.shift(1))

df_returns = df_returns.dropna()

df_returns.reset_index(inplace=True)

print(df_returns.shape)
df_returns.head()

(0, 505)


Ticker,Date,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC


In [11]:
def rolling_data(df, window_years=2, step_years=1):
    df_copy=df.copy()
    df_copy['Year']=df_copy['Date'].dt.year
    min_date=df_copy['Year'].min()
    df_copy1=df_copy[df_copy['Year'].isin([min_date,min_date+1])]
    df_copy2=df_copy[df_copy['Year'].isin([min_date+1,min_date+2])]
    df_copy3=df_copy[df_copy['Year'].isin([min_date+2,min_date+3])]
    df_copy4=df_copy[df_copy['Year'].isin([min_date+3,min_date+4])]
    df_copy5=df_copy[df_copy['Year'].isin([min_date+4,min_date+5])]
    df_copy6=df_copy[df_copy['Year'].isin([min_date+5,min_date+6])]
    
    df_copy1.drop(columns=['Date','Year'],inplace=True)
    df_copy2.drop(columns=['Date','Year'],inplace=True)
    df_copy3.drop(columns=['Date','Year'],inplace=True)
    df_copy4.drop(columns=['Date','Year'],inplace=True)
    df_copy5.drop(columns=['Date','Year'],inplace=True)
    df_copy6.drop(columns=['Date','Year'],inplace=True)

    return df_copy1,df_copy2,df_copy3,df_copy4,df_copy5,df_copy6

data=rolling_data(df_returns, window_years=2, step_years=1)
df1,df2,df3,df4,df5,df6=data[0],data[1],data[2],data[3],data[4],data[5]

In [12]:
df1

Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC


In [13]:
df6

Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC


In [14]:
def compute_risk_measures(returns_series,index_returns,THRESHOLD_Q,Q_LEVEL):
    cum_ret = returns_series.cumsum()
    drawdowns = cum_ret.expanding().max() - cum_ret

    edor = np.mean(np.maximum(drawdowns - THRESHOLD_Q, 0))

    cdar = np.mean(np.sort(drawdowns)[-int(Q_LEVEL * len(drawdowns)):])
    
    cvar_threshold = np.quantile(returns_series, 1-Q_LEVEL)
    cvar = -returns_series[returns_series <= cvar_threshold].mean()

    lpm1 = np.mean(np.maximum(0 - returns_series, 0))
    lpm2 = np.mean(np.maximum(0 - returns_series, 0) ** 2)
    lpm3 = np.mean(np.maximum(0 - returns_series, 0) ** 3)
    lpm4 = np.mean(np.maximum(0 - returns_series, 0) ** 4)

    max_drawdown = drawdowns.max()
    var = np.quantile(returns_series, 1-Q_LEVEL)
    volatility = returns_series.std() * np.sqrt(252)

    mask = index_returns < 0
    index_down = index_returns[mask]
    stock_down = returns_series[mask]
    combined = pd.concat([stock_down, index_down], axis=1).dropna()
    cov = np.cov(combined.iloc[:, 0], combined.iloc[:, 1])[0, 1]
    varr = np.var(combined.iloc[:, 1])
    downside_beta=cov / varr

    return [edor, cdar, cvar, lpm1, lpm2, lpm3, lpm4, max_drawdown, var ,volatility, downside_beta ]


In [15]:
def risk_matrix(df,THRESHOLD_Q, Q_LEVEL):
    stocks=list(df.columns)
    stocks.remove('^GSPC')
    new_df = pd.DataFrame(
    {ticker: compute_risk_measures(df[ticker],df['^GSPC'],THRESHOLD_Q,Q_LEVEL) for ticker in stocks}).T

    new_df.columns = ["EDoR", "CDaR", "CVaR", "LPM1", "LPM2", "LPM3", "LPM4", "Max Drawdown","VaR","Volatility","Downside Beta"]


    return  new_df

In [16]:
THRESHOLD_Q = 0.2  
Q_LEVEL = 0.1 

risk_matrix_df1=risk_matrix(df1,THRESHOLD_Q, Q_LEVEL)
risk_matrix_df2=risk_matrix(df2,THRESHOLD_Q, Q_LEVEL)
risk_matrix_df3=risk_matrix(df3,THRESHOLD_Q, Q_LEVEL)
risk_matrix_df4=risk_matrix(df4,THRESHOLD_Q, Q_LEVEL)
risk_matrix_df5=risk_matrix(df5,THRESHOLD_Q, Q_LEVEL)
risk_matrix_df6=risk_matrix(df6,THRESHOLD_Q, Q_LEVEL)


C:\Users\Ajeet Rai\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Ajeet Rai\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


IndexError: index -1 is out of bounds for axis 0 with size 0

In [ ]:
risk_matrix_df1

In [ ]:
risk_matrix_df6

In [ ]:
rank_df1=risk_matrix_df1.rank()
rank_df1['Year']='2018'

rank_df2=risk_matrix_df2.rank()
rank_df2['Year']='2019'

rank_df3=risk_matrix_df3.rank()
rank_df3['Year']='2020'

rank_df4=risk_matrix_df4.rank()
rank_df4['Year']='2021'

rank_df5=risk_matrix_df5.rank()
rank_df5['Year']='2022'

rank_df6=risk_matrix_df6.rank()
rank_df6['Year']='2023'

final_df=pd.concat([rank_df1,rank_df2,rank_df3,rank_df4,rank_df5,rank_df6],axis=0).reset_index()
print(final_df.shape)
final_df.head()

In [ ]:
final_df[final_df['Year']=='2018']

# 3. Extreme Events: How do these rankings behave during market stress (e.g., COVID-19 crash in2020)? Are tail-focused measures like LPM4 or CVaR more consistent predictors of losses in such periods?

In [ ]:
d1 = final_df[final_df['Year'] == '2018']
d2 = final_df[final_df['Year'] == '2019']

var_corr, _ = spearmanr(d1['VaR'], d2['VaR'])
lpm3_corr, _ = spearmanr(d1['LPM3'], d2['LPM3'])

fig, ax = plt.subplots(1, 2, figsize=(14, 6))

ax[0].scatter(d1['VaR'], d2['VaR'], color='green', s=15)
ax[0].set_xlabel("VaR Rank (2018-2019)")
ax[0].set_ylabel("VaR Rank (2019-2020)")
ax[0].set_title("VaR Rank Sensitivity")
ax[0].grid(True, alpha=0.5)
ax[0].text(0.5, 1.05, f"Spearman ρ = {var_corr:.2f}", transform=ax[0].transAxes,ha='center', va='bottom', fontsize=12)

ax[1].scatter(d1['LPM3'], d2['LPM3'], color='red', s=15)
ax[1].set_xlabel("LPM3 Rank (2018-2019)")
ax[1].set_ylabel("LPM3 Rank (2019-2020)")
ax[1].set_title("LPM3 Rank Sensitivity")
ax[1].grid(True, alpha=0.5)
ax[1].text(0.5, 1.05, f"Spearman ρ = {lpm3_corr:.2f}", transform=ax[1].transAxes,ha='center', va='bottom', fontsize=12)

plt.tight_layout()
plt.show()


In [ ]:
d1 = final_df[final_df['Year'] == '2018']
d2 = final_df[final_df['Year'] == '2023']

var_corr, _ = spearmanr(d1['VaR'], d2['VaR'])
lpm3_corr, _ = spearmanr(d1['LPM3'], d2['LPM3'])

fig, ax = plt.subplots(1, 2, figsize=(14, 6))

ax[0].scatter(d1['VaR'], d2['VaR'], color='green', s=15)
ax[0].set_xlabel("VaR Rank (2018-2019)")
ax[0].set_ylabel("VaR Rank (2023-2014)")
ax[0].set_title("VaR Rank Sensitivity")
ax[0].grid(True, alpha=0.5)
ax[0].text(0.5, 1.05, f"Spearman ρ = {var_corr:.2f}", transform=ax[0].transAxes,ha='center', va='bottom', fontsize=12)

ax[1].scatter(d1['LPM3'], d2['LPM3'], color='red', s=15)
ax[1].set_xlabel("LPM3 Rank (2018-2019)")
ax[1].set_ylabel("LPM3 Rank (2023-2024)")
ax[1].set_title("LPM3 Rank Sensitivity")
ax[1].grid(True, alpha=0.5)
ax[1].text(0.5, 1.05, f"Spearman ρ = {lpm3_corr:.2f}", transform=ax[1].transAxes,ha='center', va='bottom', fontsize=12)

plt.tight_layout()
plt.show()


In [ ]:
d1 = final_df[(final_df['Year'] == '2018') & (final_df['CVaR'] > 400)]
d2 = final_df[(final_df['Year'] == '2019') & (final_df['CVaR'] > 400)]

d3 = final_df[(final_df['Year'] == '2018') & (final_df['LPM3'] > 400)]
d4 = final_df[(final_df['Year'] == '2019') & (final_df['LPM3'] > 400)]

var_corr, _ = spearmanr(d1['CVaR'], d2['CVaR'])
lpm3_corr, _ = spearmanr(d3['LPM3'], d4['LPM3'])

fig, ax = plt.subplots(1, 2, figsize=(14, 6))

ax[0].scatter(d1['CVaR'], d2['CVaR'], color='green', s=15)
ax[0].set_xlabel("CVaR Rank (2018-2019)")
ax[0].set_ylabel("CVaR Rank (2019-2020)")
ax[0].set_title("CVaR Rank Sensitivity")
ax[0].axhline(y=400, color='blue', linestyle='--', linewidth=1)
ax[0].axvline(x=400, color='blue', linestyle='--', linewidth=1)
ax[0].grid(True, alpha=0.5)
ax[0].text(0.5, 1.05, f"Spearman ρ = {var_corr:.2f}", transform=ax[0].transAxes,
           ha='center', va='bottom', fontsize=12)

ax[1].scatter(d3['LPM3'], d4['LPM3'], color='red', s=15)
ax[1].set_xlabel("LPM3 Rank (2018-2019)")
ax[1].set_ylabel("LPM3 Rank (2019-2020)")
ax[1].set_title("LPM3 Rank Sensitivity")
ax[1].axhline(y=400, color='blue', linestyle='--', linewidth=1)
ax[1].axvline(x=400, color='blue', linestyle='--', linewidth=1)
ax[1].grid(True, alpha=0.5)
ax[1].text(0.5, 1.05, f"Spearman ρ = {lpm3_corr:.2f}", transform=ax[1].transAxes,
           ha='center', va='bottom', fontsize=12)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

d1 = final_df[(final_df['Year'] == '2018') & (final_df['CVaR'] > 400)]
d2 = final_df[(final_df['Year'] == '2023') & (final_df['CVaR'] > 400)]

d3 = final_df[(final_df['Year'] == '2018') & (final_df['LPM3'] > 400)]
d4 = final_df[(final_df['Year'] == '2023') & (final_df['LPM3'] > 400)]

var_corr, _ = spearmanr(d1['CVaR'], d2['CVaR'])
lpm3_corr, _ = spearmanr(d3['LPM3'], d4['LPM3'])

fig, ax = plt.subplots(1, 2, figsize=(14, 6))

ax[0].scatter(d1['CVaR'], d2['CVaR'], color='green', s=15)
ax[0].set_xlabel("CVaR Rank (2018-2019)")
ax[0].set_ylabel("CVaR Rank (2023-2024)")
ax[0].set_title("CVaR Rank Sensitivity")
ax[0].axhline(y=400, color='blue', linestyle='--', linewidth=1)
ax[0].axvline(x=400, color='blue', linestyle='--', linewidth=1)
ax[0].grid(True, alpha=0.5)
ax[0].text(0.5, 1.05, f"Spearman ρ = {var_corr:.2f}", transform=ax[0].transAxes,
           ha='center', va='bottom', fontsize=12)

ax[1].scatter(d3['LPM3'], d4['LPM3'], color='red', s=15)
ax[1].set_xlabel("LPM3 Rank (2018-2019)")
ax[1].set_ylabel("LPM3 Rank (2023-2024)")
ax[1].set_title("LPM3 Rank Sensitivity")
ax[1].axhline(y=400, color='blue', linestyle='--', linewidth=1)
ax[1].axvline(x=400, color='blue', linestyle='--', linewidth=1)
ax[1].grid(True, alpha=0.5)
ax[1].text(0.5, 1.05, f"Spearman ρ = {lpm3_corr:.2f}", transform=ax[1].transAxes,
           ha='center', va='bottom', fontsize=12)

plt.tight_layout()
plt.show()


In [ ]:
high_risk1 = final_df[(final_df['Year'] == '2018') & (final_df['CVaR'] > 400)]['index']
high_risk2 = final_df[(final_df['Year'] == '2020') & (final_df['CVaR'] > 400)]['index']

high_risk1

In [ ]:
data=df_returns[(df_returns['Date']>='2018-01-01') & (df_returns['Date']<='2021-12-31')]

In [ ]:
df_returns.head()

In [ ]:
d1 = final_df[(final_df['Year'] == '2018') & (final_df['CVaR'] > 400)]
x = d1['index'].unique()  # Get unique stock identifiers

d2 = final_df[(final_df['Year'] == '2023') & (final_df['index'].isin(x))]

d3 = final_df[(final_df['Year'] == '2018') & (final_df['LPM3'] > 400)]
y = d3['index'].unique()

d4 = final_df[(final_df['Year'] == '2023') & (final_df['index'].isin(y))]


In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

# Identify high-risk stocks in 2018
high_cvar_2018 = final_df[(final_df['Year'] == '2018') & (final_df['CVaR'] > 400)]['index'].unique()
high_lpm3_2018 = final_df[(final_df['Year'] == '2018') & (final_df['LPM3'] > 400)]['index'].unique()

# Get all 2023 data
cvar_2023 = final_df[final_df['Year'] == '2023']
lpm3_2023 = final_df[final_df['Year'] == '2023']

# Split into highlighted and non-highlighted sets
highlight_cvar = cvar_2023[cvar_2023['index'].isin(high_cvar_2018)]
other_cvar = cvar_2023[~cvar_2023['index'].isin(high_cvar_2018)]

highlight_lpm3 = lpm3_2023[lpm3_2023['index'].isin(high_lpm3_2018)]
other_lpm3 = lpm3_2023[~lpm3_2023['index'].isin(high_lpm3_2018)]

# Correlation using only highlighted stocks
var_corr, _ = spearmanr(highlight_cvar['CVaR'], highlight_cvar['CVaR'])  # Dummy to keep label
lpm3_corr, _ = spearmanr(highlight_lpm3['LPM3'], highlight_lpm3['LPM3'])

# Plot
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# CVaR plot
ax[0].scatter(other_cvar['CVaR'], other_cvar['CVaR'], color='lightgray', s=15, label='Other Stocks')
ax[0].scatter(highlight_cvar['CVaR'], highlight_cvar['CVaR'], color='green', s=30, label='High CVaR in 2018')

ax[0].set_xlabel("CVaR (2023)")
ax[0].set_ylabel("CVaR (2023)")
ax[0].set_title("CVaR Highlight: High-Risk in 2018")
ax[0].axhline(y=400, color='blue', linestyle='--', linewidth=1)
ax[0].axvline(x=400, color='blue', linestyle='--', linewidth=1)
ax[0].grid(True, alpha=0.5)
ax[0].legend()

# LPM3 plot
ax[1].scatter(other_lpm3['LPM3'], other_lpm3['LPM3'], color='lightgray', s=15, label='Other Stocks')
ax[1].scatter(highlight_lpm3['LPM3'], highlight_lpm3['LPM3'], color='red', s=30, label='High LPM3 in 2018')

ax[1].set_xlabel("LPM3 (2023)")
ax[1].set_ylabel("LPM3 (2023)")
ax[1].set_title("LPM3 Highlight: High-Risk in 2018")
ax[1].axhline(y=400, color='blue', linestyle='--', linewidth=1)
ax[1].axvline(x=400, color='blue', linestyle='--', linewidth=1)
ax[1].grid(True, alpha=0.5)
ax[1].legend()

plt.tight_layout()
plt.show()
